In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import warnings
warnings.filterwarnings('ignore')

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 
                                                                           100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [7]:
    batch_size = 64
    test_batch_size = 1000
    epochs = 10
    lr = 0.01
    momentum = 0.5
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
    train_loader = torch.utils.data.DataLoader(datasets.MNIST('', train=True, download=True, 
                    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])),batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(datasets.MNIST('', train=False, 
                    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])),batch_size=test_batch_size, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

100%|██████████| 9912422/9912422 [00:02<00:00, 3599647.38it/s]


Extracting MNIST\raw\train-images-idx3-ubyte.gz to MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28952125.67it/s]

Extracting MNIST\raw\train-labels-idx1-ubyte.gz to MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3391587.59it/s]


Extracting MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4482477.36it/s]


Extracting MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST\raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.322062
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.310181
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.557661
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.548077
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.252929
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.316992
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.162737
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.117547
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.238822
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.272377
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.296477
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.105239
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.276948
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.080731
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.117213
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.128169
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.055346
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.081304
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.08860